In [1]:
# import libraries

import openeo
import folium
import json
import shapely.geometry
from openeo.processes import ProcessBuilder
import os


# connect to openeo
conn = openeo.connect("https://openeo.vito.be").authenticate_oidc()

# out dir
outdir = "./input/senales"

# aoi
aoi = json.load(open('input/senales/senales_wgs84.geojson'))

# check the aoi
region = aoi['features'][0]['geometry']
geom = shapely.geometry.shape(region)
centroid = geom.centroid
center_latlon = [centroid.y, centroid.x]

m = folium.Map(location=center_latlon, zoom_start=9)

folium.GeoJson(aoi).add_to(m)
m

# define time period
time_period = ['2024-04-02', '2024-04-02']

Authenticated using refresh token.


In [2]:
s2 = conn.load_collection(
    'SENTINEL2_L1C',
    spatial_extent=region,
    temporal_extent=time_period,
    bands=["B02", "B03", "B04", "B08", "B11"])
 
s2_L2A = conn.load_collection(
    'SENTINEL2_L2A',
    spatial_extent=region,
    temporal_extent=time_period,
    bands=['SCL'])

worldcover = conn.load_collection(
    'ESA_WORLDCOVER_10M_2021_V2',
    spatial_extent=region,
    bands=['MAP']).resample_spatial(projection=32632)

dem = conn.load_collection(
    "COPERNICUS_30",
    spatial_extent=region,
    bands=["DEM"]).resample_spatial(projection=32632)

water_mask = (worldcover == 80).reduce_dimension(dimension="t", reducer="mean")

s2 = s2.merge_cubes(s2_L2A).merge_cubes(water_mask)

green = s2.band("B03")
swir = s2.band("B11")
nir = s2.band("B08")
scl = s2.band("SCL")
water = s2.band("MAP")

# NDSI
ndsi = (green - swir) / (green + swir)

# cloud mask
cloud_mask = ( (scl == 8) | (scl == 9) | (scl == 3) | (scl == 10) ) * 1.0 # times one forces to binary


In [3]:
valid_mask = (~cloud_mask) & (water !=1)

In [4]:
# valid_mask.download(os.path.join(outdir,'valid_mask.nc'))

In [5]:
snow_sure = (ndsi > 0.6) & (nir > 0.45) & valid_mask
no_snow_sure = (ndsi < 0) & valid_mask

In [6]:
# snow_sure.download(os.path.join(outdir,'snow_sure.nc'))
# no_snow_sure.download(os.path.join(outdir,'no_snow_sure.nc'))

In [7]:
# Combine to a snow_map: 0 = uncertain, 1 = sure no-snow, 2 = sure snow
snow_map = snow_sure.multiply(2) + no_snow_sure.multiply(1)

In [8]:
# snow_map.download(os.path.join(outdir,'snow_map.nc'))

In [9]:
# # distance 

# distance_udf = openeo.UDF.from_file("distance_udf.py")

# distance = snow_sure.apply_neighborhood(process=distance_udf,
#                                         size=[{"dimension": "x", "value": 2048, "unit": "px"},
#                                               {"dimension": "y", "value": 2048, "unit": "px"},],
#                                         overlap=[{"dimension": "x", "value": 1024, "unit": "px"},
#                                                  {"dimension": "y", "value": 1024, "unit": "px"}])

The computed distance looks a bit strange. In the upper left and lower right corner, the distance starts to decrease even though there is no snow in that area? 

In [10]:
# distance.download('input/distance_normalized.nc')

### distance

In [11]:
# # distance 

# distance_udf = openeo.UDF.from_file("distance_udf.py")

# distance = snow_sure.apply_neighborhood(process=distance_udf,
#                                         size=[{"dimension": "x", "value": 512, "unit": "px"},
#                                               {"dimension": "y", "value": 512, "unit": "px"},],
#                                         overlap=[{"dimension": "x", "value": 256, "unit": "px"},
#                                                  {"dimension": "y", "value": 256, "unit": "px"}]) 

In [12]:
# distance = distance.save_result(format="NetCDF")
# job = distance.create_job()
# job.start_and_wait()
# job.download_results("distance/")

In [13]:
# # normalize distance to range 0f 0-1
# distance_min = distance.reduce_dimension("t", reducer="min")
# distance_max = distance.reduce_dimension("t", reducer="max")

# normalized_distance = (distance - distance_min) / (distance_max - distance_min)


In [14]:
# # normalized_distance.download('input/normalized_distance.nc')
# normalized_distance = normalized_distance.save_result(format="NetCDF")
# job = distance.create_job()
# job.start_and_wait()
# job.download_results("input/")

Here I decided to switch to single date datacube instead of the month I previously had. Basically all the problems so far where because of the multiple time steps. And now the final straw was that masking the dem with snow_sure would require the dem also to have the same time steps as the snow_sure. I tried to add them but couldn't figure it out. So only one day now instead of multiple days.

### mask dem

In [15]:
snow_mask = snow_sure.reduce_dimension("t", reducer="mean") # snowmap and dem have different time values in 't', so reduce 't'
# snow_mask.download(os.path.join(outdir,'snowmask.nc'))

In [16]:
dem = dem.reduce_dimension(dimension='t', reducer='mean')
# dem.download(os.path.join(outdir,'dem.nc'))

In [17]:
masked_dem = dem.mask(snow_sure==0) # get dem for confident snow areas only
# masked_dem.download(os.path.join(outdir,'masked_dem.nc'))

In [18]:
# print(masked_dem.metadata)

In [19]:
# convert snow_sure to polygon
# snow_polygon = snow_sure.mask(snow_sure == 0).raster_to_vector()
# snow_polygon.download(os.path.join(outdir, 'snow_polygon.geojson'))

In [20]:
print(snow_polygon.metadata)

None


#### align dem time with snow_sure

In [21]:
snow_sure = snow_sure.add_dimension(name="bands", label="snow_sure", type="bands")
dem = dem.rename_labels("bands", ["DEM"]) 
combined = dem.merge_cubes(snow_sure)

# combined = dem.rename_labels("bands", ["DEM"]).merge_cubes(
#     masked_dem.rename_labels("bands", ["masked_DEM"])
# )


In [22]:
print(combined.metadata)

CollectionMetadata({'spatial': {'bbox': [[-180.0013889, -89.9998611, 179.9986111, 84.0001389]]}, 'temporal': {'interval': [['2010-12-12T00:00:00Z', None]]}} - ['DEM', 'snow_sure'] - ['bands', 't', 'x', 'y'])


In [23]:
snow_poly = json.load(open('input/senales/snow_polygon.geojson'))

import json
from shapely.geometry import shape, mapping
from pyproj import Transformer
from shapely.ops import transform
project = Transformer.from_crs("EPSG:32632", "EPSG:4326", always_xy=True).transform

# Apply transformation
geom = shape(snow_poly["features"][0]["geometry"])  # assumes one feature
geom_wgs84 = transform(project, geom)

# Convert back to GeoJSON format
snow_poly_wgs84 = {
    "type": "FeatureCollection",
    "features": [{
        "type": "Feature",
        "geometry": mapping(geom_wgs84),
        "properties": {}
    }]}

In [51]:
altitude_mask_udf = openeo.UDF.from_file("altitude_mask_udf.py")
altitude_mask = combined.apply_polygon(geometries=snow_poly_wgs84, process=altitude_mask_udf)

In [52]:
# this one runs but gives boxy look due to chunking
# altitude_mask_udf = openeo.UDF.from_file("altitude_mask_udf.py")

# altitude_mask = combined.apply_neighborhood(process=altitude_mask_udf,
#                                         size=[{"dimension": "x", "value": 256, "unit": "px"},
#                                               {"dimension": "y", "value": 256, "unit": "px"},],
#                                         overlap=[{"dimension": "x", "value": 128, "unit": "px"},
#                                                  {"dimension": "y", "value": 128, "unit": "px"}])

In [53]:
# altitude_mask = altitude_mask.save_result(format="NetCDF")
# job = altitude_mask.create_job()
# job.start_and_wait()
# job.download_results(outdir)

In [54]:
altitude_mask.download(os.path.join(outdir, 'altitude_mask.nc'))

i get the chunking problem again with the altitude mask. i tried with apply and get very chunky results. then tried apply_neighborhood and results are less chunky but still chunky and again I would have the problem of increasing the size and overlap in larger areas

### index of distance

In [25]:
# 5. Combine normalized and altitude into index of distance
index_of_distance = normalized_distance * altitude_mask

In [26]:
index_of_distance.download('input/distance_index.nc')

In [27]:
index_scaled = index_of_distance.linear_scale_range(0, 1, 0, 254).round()

AttributeError: 'DataCube' object has no attribute 'round'

In [ ]:
def adiacency_indexes(curr_acquisition, curr_aux_folder, auxiliary_folder_path, no_data_mask, bands):
    sensor = get_sensor(os.path.basename(curr_acquisition))
    
    path_cloud_mask = glob.glob(os.path.join(curr_aux_folder, '*cloud_mask.nc'))[0]
    path_water_mask = glob.glob(os.path.join(auxiliary_folder_path, '*water_mask.nc'))[0]
    NDSI_path = glob.glob(os.path.join(curr_aux_folder, '*ndsi.nc'))[0]
    dem_path = glob.glob(os.path.join(auxiliary_folder_path, '*dem.nc'))[0]
    
    valid_mask = np.logical_not(no_data_mask)
    
    # Load masks and other necessary data
    cloud_mask, curr_image_info = open_image(path_cloud_mask)
    water_mask = open_image(path_water_mask)[0]
    curr_scene_valid = np.logical_not(np.logical_or.reduce((cloud_mask == 2, water_mask == 1, no_data_mask)))
    dem = open_image(dem_path)[0]
    NDSI = open_image(NDSI_path)[0]
    NIR = bands['NIR']
    
    # Create the snow map
    snow_map = np.zeros_like(NDSI, dtype=np.uint8)
    no_snow_sure = (NDSI < 0) & curr_scene_valid
    snow_sure = (NDSI > 0.6) & (NIR > 0.45) & curr_scene_valid
    snow_map[no_snow_sure] = 1
    snow_map[snow_sure] = 2

    # Calculate distance from snow_sure
    distance_from_snow = np.full_like(snow_map, np.nan, dtype=np.float32)
    snow_sure_pixels = (snow_map == 2)
    distance_from_snow[curr_scene_valid] = distance_transform_edt(~snow_sure_pixels)[curr_scene_valid]
    distance_from_snow = np.nan_to_num(distance_from_snow, nan=np.nanmax(distance_from_snow))
    distance_from_snow_normalized = (distance_from_snow - np.nanmin(distance_from_snow)) / (
        np.nanmax(distance_from_snow) - np.nanmin(distance_from_snow)
    )
    
    # Set altitude threshold
    valid_dem = dem[np.logical_and(curr_scene_valid, snow_map == 2)]
    
    if valid_dem.size > 0:
        altitude_min_threshold = np.percentile(valid_dem, 1) - 200
    else:
        altitude_min_threshold = np.nan  # Oppure scegli un valore predefinito sensato
    
    altitude_mask = (dem >= altitude_min_threshold) if not np.isnan(altitude_min_threshold) else np.zeros_like(dem, dtype=bool)


    
    altitude_mask = (dem >= altitude_min_threshold)
    
    # Combine distance and altitude into index_of_distance
    index_of_distance = np.zeros_like(snow_map, dtype=np.float32)
    index_of_distance[curr_scene_valid] = (
        distance_from_snow_normalized[curr_scene_valid] * altitude_mask[curr_scene_valid]
    )
    
   
    # Convert to uint8 for saving
    index_of_distance_uint8 = (index_of_distance * 254).astype(np.uint8)  # Scale if needed
    
    # Set no-data value for areas outside altitude_mask
    no_data_value = 255  # Choose the no-data value, e.g., 0 or 255
    index_of_distance_uint8[np.logical_or(~altitude_mask, ~curr_scene_valid)] = no_data_value

    
    # Save the result as a GeoTIFF
    output_path = os.path.join(curr_aux_folder, "index_of_distance.tif")
    transform = from_origin(curr_image_info['geotransform'][0], curr_image_info['geotransform'][3], 
                            curr_image_info['geotransform'][1], -curr_image_info['geotransform'][5])
    with rasterio.open(
        output_path,
        "w",
        driver="GTiff",
        height=index_of_distance_uint8.shape[0],
        width=index_of_distance_uint8.shape[1],
        count=1,
        dtype=rasterio.uint8,
        crs=curr_image_info['projection'],
        transform=transform,
        nodata=no_data_value,
    ) as dst:
        dst.write(index_of_distance_uint8, 1)